## Language Similarity
This is a notebook to compute the word embeddings for each week of the May and June 2022 subreddits for the cross sectional analysis.

In [1]:
pip install pyarrow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 40.7 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 84.0 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
#Download the SBERT model to deploy on the 'body' and the 'receiver_body' columns, so that the word embeddings can be obtained.
!pip install -U sentence-transformers
# load tqdm
!pip install --force https://github.com/chengs/tqdm/archive/colab.zip
!pip install swifter

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 21.0 MB/s eta 0:00:00
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.28.1
    Uninstalling tqdm-4.28.1:
      Successfully uninstalled tqdm-4.28.1

[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
  Using cached https://github.com/chengs/tqdm/archive/colab.zip
  Preparing metadata (setup.py) ... - done
  Created wheel for tqdm: filename=tqdm-4.28.1-py2.py3-none-any.whl size=47874 sha256=b5a92152648fdec95a70d620d820513b99cc00434984794bf17ca78c9a317dcf
  Stored in directory: /tmp/pip-ephem-wheel-cache-eycluf5i/wheels/a7/30/17/f85112bf3ff9d07da5ef0a111c02e4a86df55d990ce52864cd
Successfully built tqdm
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.66.4
    Uninstalling tqdm-4.66.4:
      Successfully uninstalled tqdm-4.66.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installe

In [5]:
import numpy as np
import pandas as pd
import nltk
from sentence_transformers import SentenceTransformer, SentencesDataset, InputExample, losses, util
from torch.utils.data import DataLoader
from sentence_transformers import losses
import os
import swifter
from nltk.tokenize import sent_tokenize
import torch
nltk.download('punkt')
from tqdm import tqdm
tqdm.pandas()
import pickle

[nltk_data] Downloading package punkt to /home/datalore/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [6]:
from sentence_transformers import SentenceTransformer
sbert_model = SentenceTransformer('all-mpnet-base-v2')

In [7]:
def generate_language_similarity(val1, val2):
    return util.dot_score(val1, val2)

#define function to generate embeddings for one row
def generate_embeddings(record):
   return sbert_model.encode(record, convert_to_numpy=True, normalize_embeddings=True)

def divide_data_3_parts(week_data):
    week_data.head(3)
    div, mod = divmod(len(week_data), 3)
    part1 = week_data[:div]
    part2 = week_data[div:div * 2]
    part3 = week_data[div * 2:]
    if mod == 1:
        part3 = pd.concat([part3, part2.iloc[-1:]], ignore_index=True)
        part2 = part2.iloc[:-1]
    elif mod == 2:
        part3 = pd.concat([part3, part2.iloc[-2:]], ignore_index=True)
        part2 = part2.iloc[:-2]
    return part1, part2, part3

def word_embeddings(week_no,part_data,part_no):
    #call the function to generate word embeddings
    print('start generating the word embeddings')
    body_embeddings = part_data['body'].progress_apply(generate_embeddings)
    print('finished generation of body embeddings')
    print(len(body_embeddings))
    receiver_body_embeddings = part_data['receiver_body'].progress_apply(generate_embeddings)
    print('finished generation of receiver embeddings')
    print(len(receiver_body_embeddings))
    print('finished generation of word embeddings')
    model_storage_name_location = 'word_embeddings/embeddings_w'+ str(week_no) +'_part'+str(part_no)+ '.pkl'
    # Open the file in binary write mode
    with open(model_storage_name_location, 'wb') as f:
        pickle.dump(body_embeddings, f)
    print(type(body_embeddings))
    model_storage_name_location_receiver = 'word_embeddings/receiver_embeddings_w'+ str(week_no) +'_part'+str(part_no)+ '.pkl'
    # Open the file in binary write mode
    with open(model_storage_name_location_receiver, 'wb') as f:
        pickle.dump(receiver_body_embeddings, f)
    print(type(receiver_body_embeddings))
    #compute the languageSimilarity
    df1 = pd.concat([body_embeddings, receiver_body_embeddings], axis=1)
    print(len(df1))
    df1['languageSimilarity_commentLevel'] = df1.progress_apply(lambda x: (util.dot_score(x[0],x[1])).item(), axis=1)
    extracted_col = df1['languageSimilarity_commentLevel']
    print(len(extracted_col))
    part_data_final = pd.concat([part_data, extracted_col], axis=1)
    print(len(part_data_final))
    #part_data_final.head(3)
    resulting_file_name = 'bothSim_Processed/mayjune_w'+str(week_no)+'_part'+str(part_no)+ '_processed.parquet'
    part_data_final.to_parquet(resulting_file_name)
    print('completed!')

In [7]:
##############################
#read the embeddings
# week = 1
# part = 3
# body_embeddings_1 = pd.read_pickle('JanFebSubreddits2022/word_embeddings/embeddings_w'+str(week)+'_part'+str(part)+'.pkl')
# print(len(body_embeddings_1))
# print(body_embeddings_1[0])
# print(type(body_embeddings_1))
# receiver_embeddings_1 = pd.read_pickle('JanFebSubreddits2022/word_embeddings/receiver_embeddings_w'+str(week)+'_part'+str(part)+'.pkl')
# print(len(receiver_embeddings_1))
# print(receiver_embeddings_1[0])
# print(type(receiver_embeddings_1))

In [8]:
#############################
#read the processed file with both network similarity and language similarity
# week = 1
# part = 3
# file_path1 = 'JanFebSubreddits2022/bothSim_Processed/janfeb_w' + str(week) + '_part'+str(part)+'_processed.parquet'
# week_data2 = pd.read_parquet(file_path1)
# print(week_data2['subreddit'].value_counts())
# print(week_data2['date'].value_counts())
# week_data2.head(2)

## Process Week 18.

In [8]:
#reading the data and dividing it into parts
w18 = pd.read_parquet('netSim_processed/mayjune_w18_processed.parquet')
print(len(w18))
w18_p1,w18_p2,w18_p3 = divide_data_3_parts(w18)
print(len(w18_p1))
print(len(w18_p2))
print(len(w18_p3))
print(len(w18_p1)+len(w18_p2)+len(w18_p3))
w18['subreddit'].value_counts()

1008518
336172
336170
336176
1008518


amitheasshole 372661
politics 257921
worldnews 230104
news 118196
science 29636
Name: subreddit, dtype: int64

In [9]:
w18['date'].value_counts()

2022-05-03 222768
2022-05-04 162014
2022-05-02 144216
2022-05-05 140083
2022-05-06 139102
2022-05-07 117116
2022-05-01 83219
Name: date, dtype: int64

In [10]:
w18.head(2)

,id,subreddit,body,author,parent_id,link_id,receiver,receiver_body,date,toxicityScore,identityAttackScore,insultScore,obsceneScore,severeToxicityScore,threatScore,week,networkSimilarity
0,i6ukmz0,politics,Fingers crossed. Get your passports arranged,barflyondabeach,i6uki4c,ufc7kc,platinumtaters7,I can't wait for the collapse. Sane Americans ...,2022-05-01,0.001366,0.000158,0.000198,0.000174,0.000096,0.000125,week18,0.525826
1,i6ukp3m,amitheasshole,^^^^AUTOMOD ***Thanks for posting! This comme...,automoderator,ufngd4,ufngd4,disastrous_length_,WIBTA If i told my uncle he cant move in with ...,2022-05-01,0.026031,0.000327,0.001232,0.002025,0.000153,0.000213,week18,0.352773


In [14]:
word_embeddings(18,w18_p1,1)

start generating the word embeddings
finished generation of body embeddings
336172
finished generation of receiver embeddings
336172
finished generation of word embeddings
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
336172
336172
336172
completed!


100%|██████████| 336172/336172 [00:17<00:00, 19733.71it/s]


In [24]:
#read the processed file with both network similarity and language similarity
week = 18
part = 1
file_path1 = 'bothSim_Processed/mayjune_w' + str(week) + '_part'+str(part)+'_processed.parquet'
test = pd.read_parquet(file_path1)
test.head(2)

,id,subreddit,body,author,parent_id,link_id,receiver,receiver_body,date,toxicityScore,identityAttackScore,insultScore,obsceneScore,severeToxicityScore,threatScore,week,networkSimilarity,languageSimilarity_commentLevel
0,hyuhzka,amitheasshole,^^^^AUTOMOD ***Thanks for posting! This comme...,automoderator,t3tuvk,t3tuvk,hawoona,WIBTA if I told my bestfriend I'm upset she ke...,2022-03-01,0.019431,0.000259,0.001074,0.001195,0.000123,0.000201,week9,0.277044,0.604219
1,hyui15e,worldnews,"Internet's good, it just says my portfolio is ...",decomposition_,hyuhy3k,t3rvl4,bokidge,Bro check your internet first lol,2022-03-01,0.000587,0.000143,0.000175,0.000184,0.000126,0.000132,week9,0.383257,0.371359


In [11]:
word_embeddings(18,w18_p2,2)

start generating the word embeddings
finished generation of body embeddings
336170
finished generation of receiver embeddings
336170
finished generation of word embeddings
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
336170
336170
336170
completed!


100%|██████████| 336170/336170 [00:16<00:00, 20073.10it/s]


In [12]:
#read the processed file with both network similarity and language similarity
week = 18
part = 2
file_path1 = 'bothSim_Processed/mayjune_w' + str(week) + '_part'+str(part)+'_processed.parquet'
test = pd.read_parquet(file_path1)
test.head(2)

,id,subreddit,body,author,parent_id,link_id,receiver,receiver_body,date,toxicityScore,identityAttackScore,insultScore,obsceneScore,severeToxicityScore,threatScore,week,networkSimilarity,languageSimilarity_commentLevel
336172,i75mqby,politics,"&gt; North Carolina\n\nAt the moment, we have ...",sydbobyd,i75ae24,uhdf00,jdoreh,"&gt;The Center for Reproductive Rights, a grou...",2022-05-03,0.000988,0.000158,0.000176,0.000179,0.000104,0.000117,week18,0.839257,0.410606
336173,i75mqhe,amitheasshole,YTA- you went too far,same_hurry8142,uhdt1f,uhdt1f,yellowribboninoak,AITA for kicking my son out of the car?,2022-05-03,0.022659,0.000309,0.000784,0.000543,0.000108,0.000218,week18,0.380447,0.248420


In [13]:
word_embeddings(18,w18_p3,3)

start generating the word embeddings
finished generation of body embeddings
336176
finished generation of receiver embeddings
336176
finished generation of word embeddings
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
336176
336176
336176
completed!


100%|██████████| 336176/336176 [00:16<00:00, 20989.72it/s]


In [ ]:
#read the processed file with both network similarity and language similarity
week = 18
part = 3
file_path1 = 'bothSim_Processed/marchapril_w' + str(week) + '_part'+str(part)+'_processed.parquet'
test = pd.read_parquet(file_path1)
test.head(2)

## Processing Week 19.

In [11]:
#reading the data and dividing it into parts
w19 = pd.read_parquet('netSim_processed/mayjune_w19_processed.parquet')
print(len(w19))
w19_p1,w19_p2,w19_p3 = divide_data_3_parts(w19)
print(len(w19_p1))
print(len(w19_p2))
print(len(w19_p3))
print(len(w19_p1)+len(w19_p2)+len(w19_p3))

950801
316933
316931
316937
950801


In [12]:
w19['subreddit'].value_counts()

amitheasshole 399228
worldnews 230103
politics 200954
news 90061
science 30455
Name: subreddit, dtype: int64

In [13]:
w19['date'].value_counts()

2022-05-11 151543
2022-05-13 144938
2022-05-12 137306
2022-05-10 133988
2022-05-09 131684
2022-05-14 129859
2022-05-08 121483
Name: date, dtype: int64

In [14]:
w19.head(2)

,id,subreddit,body,author,parent_id,link_id,receiver,receiver_body,date,toxicityScore,identityAttackScore,insultScore,obsceneScore,severeToxicityScore,threatScore,week,networkSimilarity
0,i7ql6ye,amitheasshole,Until you're legally an adult it would be disr...,neomiahsmom314,ukp51h,ukp51h,azraeltheanarchist,WIBTA To Get A Tattoo Against My Mother's Wishes?,2022-05-08,0.64323,0.008299,0.120982,0.011706,0.000897,0.003151,week19,0.882737
1,i7ql74f,amitheasshole,"Oh, I would absolutely follow through with sep...",oldmom64,i7q8itd,uknihc,treswhat,NTA. She messed up and now has to earn back mo...,2022-05-08,0.00060,0.000148,0.000185,0.000191,0.000132,0.000133,week19,0.316218


In [18]:
word_embeddings(19,w19_p1,1)

start generating the word embeddings
finished generation of body embeddings
316933
finished generation of receiver embeddings
316933
finished generation of word embeddings
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
316933
316933
316933
completed!


100%|██████████| 316933/316933 [00:13<00:00, 22665.46it/s]


In [19]:
#read the processed file with both network similarity and language similarity
week = 19
part = 1
file_path1 = 'bothSim_Processed/marchapril_w' + str(week) + '_part'+str(part)+'_processed.parquet'
test = pd.read_parquet(file_path1)
test.head(2)

,id,subreddit,body,author,parent_id,link_id,receiver,receiver_body,date,toxicityScore,identityAttackScore,insultScore,obsceneScore,severeToxicityScore,threatScore,week,networkSimilarity,languageSimilarity_commentLevel
0,hzrtvgh,amitheasshole,NTA it's not your fault your mother is manipul...,waterfairy30,t8df42,t8df42,coastfar884,AITA for not apologizing to my sister after my...,2022-03-08,0.276582,0.001487,0.038697,0.002954,0.000243,0.00053,week10,0.711669,0.556055
1,hzrtvmg,worldnews,"From what we have seen, they dont even have ad...",snowing_throwballs,hzrsn19,t92wld,aeipownu,I wonder how a similar invasion on U.S. soil w...,2022-03-08,0.000919,0.000174,0.000180,0.000170,0.000115,0.00013,week10,0.490630,0.550355


In [12]:
word_embeddings(19,w19_p2,2)

start generating the word embeddings
finished generation of body embeddings
316931
finished generation of receiver embeddings
316931
finished generation of word embeddings
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
316931
316931
316931
completed!


100%|██████████| 316931/316931 [00:15<00:00, 19943.51it/s]


In [15]:
word_embeddings(19,w19_p3,3)

start generating the word embeddings
finished generation of body embeddings
316937
finished generation of receiver embeddings
316937
finished generation of word embeddings
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
316937
316937
316937
completed!


100%|██████████| 316937/316937 [00:15<00:00, 20117.49it/s]


## Process Week 20.

In [8]:
#reading the data and dividing it into parts
w20 = pd.read_parquet('netSim_processed/mayjune_w20_processed.parquet')
print(len(w20))
w20_p1,w20_p2,w20_p3 = divide_data_3_parts(w20)
print(len(w20_p1))
print(len(w20_p2))
print(len(w20_p3))
print(len(w20_p1)+len(w20_p2)+len(w20_p3))

927040
309013
309012
309015
927040


In [9]:
w20['subreddit'].value_counts()

amitheasshole 379824
worldnews 232927
politics 188510
news 106481
science 19298
Name: subreddit, dtype: int64

In [10]:
w20['date'].value_counts()

2022-05-16 141455
2022-05-20 141099
2022-05-19 135491
2022-05-17 134099
2022-05-18 131478
2022-05-15 124434
2022-05-21 118984
Name: date, dtype: int64

In [11]:
w20.head(2)

,id,subreddit,body,author,parent_id,link_id,receiver,receiver_body,date,toxicityScore,identityAttackScore,insultScore,obsceneScore,severeToxicityScore,threatScore,week,networkSimilarity
0,i8mxzz6,worldnews,Stop trying to pretend you know anything.,maplethor,i8m9by5,upe4oe,knightofarose,If you think the fullness of intelligence rega...,2022-05-15,0.028180,0.000320,0.000785,0.000638,0.000111,0.000293,week20,0.976916
1,i8my0d9,worldnews,Jerusalem was Jewish before humanity invented ...,krneki12,i8ks9a6,upfpc7,r3miel7,Can’t wait for you to learn about Israel then,2022-05-15,0.008545,0.003734,0.000641,0.000639,0.000392,0.000378,week20,0.724038


In [13]:
word_embeddings(20,w20_p1,1)

start generating the word embeddings
finished generation of body embeddings
309013
finished generation of receiver embeddings
309013
finished generation of word embeddings
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
309013
309013
309013
completed!


100%|██████████| 309013/309013 [00:12<00:00, 23776.13it/s]


In [12]:
word_embeddings(20,w20_p2,2)

start generating the word embeddings
finished generation of body embeddings
309012
finished generation of receiver embeddings
309012
finished generation of word embeddings
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
309012
309012
309012
completed!


100%|██████████| 309012/309012 [00:15<00:00, 20485.72it/s]


In [13]:
word_embeddings(20,w20_p3,3)

start generating the word embeddings
finished generation of body embeddings
309015
finished generation of receiver embeddings
309015
finished generation of word embeddings
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
309015
309015
309015
completed!


100%|██████████| 309015/309015 [00:14<00:00, 21709.55it/s]
